In [1]:
### 融资融券对股价的影响
import h5py
import pandas as pd
from jqdatasdk import *
from datetime import datetime

auth("13818134484", "12345678")
print(get_query_count())

提示：当前环境pandas版本为0.25，get_price与get_fundamentals_continuously接口panel参数将固定为False
注意：0.25以上版本pandas不支持panel，如使用该数据结构和相关函数请注意修改
auth success 
{'total': 10000000, 'spare': 8132769}


In [44]:
# 获取交易日列表
trade_days = get_all_trade_days()
trade_days_str = [i.strftime('%Y-%m-%d') for i in list(trade_days)]

today = datetime.now()
today_str = datetime.now().strftime('%Y-%m-%d')

# 当日前一个交易日作为目标
for _, i in enumerate(trade_days_str):
    if today_str <= i:
        target_day_str = trade_days_str[_ - 1]
        break

        # 目标日与前一个交易日间隔天数
target_day_pre_delta = (
        trade_days[trade_days_str.index(target_day_str) - 1] - trade_days[trade_days_str.index(target_day_str) - 2]).days

# 获取未退市清单
stocks = get_all_securities(['stock'])
stocks['code'] = stocks.index.values

stocks['market'] = stocks[['code']].applymap(lambda x: x.split('.')[1])
stocks['stock_code'] = stocks[['code']].applymap(lambda x: x.split('.')[0])

stocks['是否退市'] = stocks[['end_date']].applymap(lambda x: 1 if x < today else 0)

stock_list = stocks[stocks['是否退市'] == 0]['code'].tolist()

In [90]:
# 特征加工
with h5py.File('D:/data_file/jqdata_bak_20210223.hdf5', 'r') as files:
    for key in list(files.keys()):
        df_list = []

        for code in list(files[key].keys()):
            tmp_list = []

            for _ in list(files[key][code].keys()):
                if _ != "price_minute":
                    columns = files[key][code][_]['columns'][:]
                    columns = ['{}@{}'.format(_, i.decode('utf-8')) for i in columns]

                    values = files[key][code][_]['values'][:]

                    tmp_df = pd.DataFrame(values)
                    tmp_df.columns = columns
                    tmp_list.append(tmp_df)

            df = pd.concat(tmp_list, axis=1)
            df_list.append(df)

AssertionError: 

In [91]:
df

,price_call@current,price_call@volume,price_call@money,price_call@a1_p,price_call@a1_v,price_call@a2_p,price_call@a2_v,price_call@a3_p,price_call@a3_v,price_call@a4_p,...,price_daily@low,price_daily@volume,price_daily@money,price_mtss@fin_value,price_mtss@fin_buy_value,price_mtss@fin_refund_value,price_mtss@sec_value,price_mtss@sec_sell_value,price_mtss@sec_refund_value,price_mtss@fin_sec_value
0,19.1,806800.0,15409880.0,19.1,73422.0,19.11,5900.0,19.12,28100.0,19.13,...,2226.99,1286929.0,2.891682e+09,3.087563e+09,298453303.0,196510766.0,102983836.0,179980.0,2133173.0,5.003062e+09


In [73]:
get_price('000001.XSHE', start_date='2021-01-04', end_date='2021-01-04', frequency='daily', fq='post')

,open,close,high,low,volume,money
2021-01-04,2306.7,2246.31,2306.7,2226.99,1286929.0,2.891682e+09
